# Лабораторная работа №4 по курсу "Интеллектуальный анализ данных".

In [3]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import sklearn.model_selection
import numpy as np
import sklearn.metrics as metr
import sklearn.pipeline
import sklearn.tree
import matplotlib.pyplot as plt
import sklearn.ensemble
import warnings
import sys  
if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [9]:
anime_csv = 'data/anime.csv'
rating = 'data/rating.csv'
data = pd.read_csv(anime_csv)

In [10]:
data.head(10)

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
5,32935,Haikyuu!!: Karasuno Koukou VS Shiratorizawa Ga...,"Comedy, Drama, School, Shounen, Sports",TV,10,9.15,93351
6,11061,Hunter x Hunter (2011),"Action, Adventure, Shounen, Super Power",TV,148,9.13,425855
7,820,Ginga Eiyuu Densetsu,"Drama, Military, Sci-Fi, Space",OVA,110,9.11,80679
8,15335,Gintama Movie: Kanketsu-hen - Yorozuya yo Eien...,"Action, Comedy, Historical, Parody, Samurai, S...",Movie,1,9.10,72534
9,15417,Gintama&#039;: Enchousen,"Action, Comedy, Historical, Parody, Samurai, S...",TV,13,9.11,81109


In [11]:
data.genre = data.genre.fillna("None")
data.type = data.type.fillna("None")
data.rating = data.rating.fillna(data.rating.mean())

In [12]:
anime = data.drop(columns=['name', 'genre', 'type'])

In [13]:
anime = pd.concat([anime,pd.get_dummies(data['name'],drop_first = True)],axis = 1)
anime = pd.concat([anime,pd.get_dummies(data['genre'],drop_first = True)],axis = 1)
anime = pd.concat([anime,pd.get_dummies(data['type'],drop_first = True)],axis = 1)

## Этап 1. Демонстрация работы KNeighborsClassifier

### Преобразование категориальных признаков в бинарные с помощью pd.factorize / pd.get_dummies

In [14]:
anime['episodes'] = pd.factorize(data['episodes'])[0]
anime['rating'] = pd.factorize(data['rating'])[0]
anime['members'] = pd.factorize(data['members'])[0]
anime

,anime_id,episodes,rating,members,"&quot;Aesop&quot; no Ohanashi yori: Ushi to Kaeru, Yokubatta Inu",&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi,&quot;Bungaku Shoujo&quot; Memoire,&quot;Bungaku Shoujo&quot; Movie,&quot;Eiji&quot;,&quot;Eiyuu&quot; Kaitai,...,Supernatural,Thriller,Vampire,Yaoi,Music,None,ONA,OVA,Special,TV
0,32281,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5114,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,28977,2,2,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,9253,3,3,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,9969,2,4,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12289,9316,0,473,4289,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
12290,5543,0,462,5845,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
12291,5621,12,409,5462,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
12292,6133,0,399,5160,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


### sklearn.preprocessing.StandardScaler

#### Нормализовать значения признаков и построить для нормализовапнного датасета KNeighborsClassifier (без использования конвееров)

In [15]:
scaler = StandardScaler()
data_tr = anime.drop("members",axis = 1)
target = anime["members"]

#### Разделение данных на тестовую и обучающую выборки.

In [16]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(data_tr, target, random_state = 0)

#### Нормализация данных

In [17]:
scaler.fit(X_train)
tdtr = scaler.transform(X_train)
tdt = scaler.transform(X_test)

#### Классификация при помощи KNeighborsClassifier

In [18]:
clf = KNeighborsClassifier(n_neighbors = 5) 
clf.fit(tdtr, y_train)
cl1 = clf.predict(tdt)

### Посмотрим долю правильных ответов

In [19]:
metr.accuracy_score(y_test, cl1) 

0.001951854261548471

### sklearn.pipeline.Pipeline

In [20]:
knn_pipe = sklearn.pipeline.Pipeline([ #инициализируем пайплайн, сначала данные попадут в scaler, потом сразу в kNN 
    ('scaler', sklearn.preprocessing.StandardScaler()), #помогает сократить код, чтоб не писать одно и тоже
    ('knn', sklearn.neighbors.KNeighborsClassifier(n_jobs=-1))
])

In [21]:
knn_pipe.fit(tdtr,y_train) #вызываемые методы должны совпадать у всех объектов пайплайна
cl2 = knn_pipe.predict(tdt)

#### Посмотрим долю правильных ответов.

In [22]:
metr.accuracy_score(y_test, cl2) #доля правильных ответов

0.001951854261548471

### Подбор гиперпараметров для KNeighborsClassifier с помощью GridSearchCV

In [23]:
knn = sklearn.neighbors.KNeighborsClassifier( n_neighbors = 10 )
#__________#
knn_params = { 'n_neighbors': range(3, 17) } 
#__________#
knn_grid = sklearn.model_selection.GridSearchCV(knn, knn_params) 
#__________#
knn_grid.fit( tdtr, y_train )

GridSearchCV(estimator=KNeighborsClassifier(n_neighbors=10),
             param_grid={'n_neighbors': range(3, 17)})

#### Получаем оптимальные параметры для заданного классификатора

In [24]:
knn_grid.best_params_, knn_grid.best_score_ 

({'n_neighbors': 11}, 0.0026030368763557484)

#### Доля правильных ответов

In [25]:
metr.accuracy_score(y_test, knn_grid.predict(tdt)) #доля правильных ответов

0.002277163305139883

## Этап 2. Демонстрация работы DecisionTreeClassifier

### Cоздаем дерево c max_depth (максимальная глубина) 3 и random_state(случайный разброс) 17

In [26]:
clf_tree = sklearn.tree.DecisionTreeClassifier( 
    criterion = 'entropy', max_depth = 3, random_state = 42
)
clf_tree.fit( tdtr, y_train )
cl_t = clf_tree.predict(tdt)
metr.accuracy_score(y_test, cl_t)

0.0035783994795055302

### Cмотрим на табличку метрик

In [28]:
print( sklearn.metrics.classification_report( 
    cl_t, y_test
) )

              precision    recall  f1-score   support

           9       0.00      0.00      0.00         0
          14       0.00      0.00      0.00         0
          16       0.00      0.00      0.00         0
          18       0.00      0.00      0.00         0
          31       0.00      0.00      0.00         0
          33       0.00      0.00      0.00         0
          36       0.00      0.00      0.00         0
          37       0.00      0.00      0.00         0
          39       0.00      0.00      0.00         0
          42       0.00      0.00      0.00         0
          44       0.00      0.00      0.00         0
          45       0.00      0.00      0.00         0
          46       0.00      0.00      0.00         0
          48       0.00      0.00      0.00         0
          50       0.00      0.00      0.00         0
          57       0.00      0.00      0.00         0
          83       0.00      0.00      0.00         0
          90       0.00    

### Произведем подбор параметров для дерева

In [29]:
tree = sklearn.tree.DecisionTreeClassifier( max_depth = 3, random_state = 42 )
tree_params = { 'max_depth': range(1, 25),
              'max_features': range(2, 20)}
tree_grid = sklearn.model_selection.GridSearchCV(tree, tree_params) 

tree_grid.fit( tdtr, y_train )

KeyboardInterrupt: 

### Посмотрим подобранные оптимальные параметры

In [ ]:
tree_grid.best_params_, tree_grid.best_score_ 

### Проверим долю правильных ответов

In [ ]:
metr.accuracy_score(y_test, tree_grid.predict(tdt)) 

### Оценим важность каждого признака для классификации

In [ ]:
clf_tree.feature_importances_ 

In [ ]:
name_col = mach.drop(columns = 'rotor').columns
name_col

In [ ]:
y = pd.Series(data=clf_tree.feature_importances_, index=name_col)
ind = y[:] == 0
y = y[~ind]

### Построить график feature_importance

In [ ]:
x = range(len(y))
plt.bar(x, y)
plt.xticks(x, y.index,rotation = 45)

### 3 наиболее важных признака в обратной сортировке

In [ ]:
y.sort_values().iloc[-4: -1]

## Этап 3. Демонстрация работы RandomForestClassifier

### Построение леса

In [30]:
import sklearn.ensemble

#количество деревьев 20
forest = sklearn.ensemble.RandomForestClassifier( n_estimators=20, random_state=17 )
forest.fit( tdtr, y_train )

RandomForestClassifier(n_estimators=20, random_state=17)

### Посмотрим на долю правильных ответов

In [31]:
metr.accuracy_score( y_test, forest.predict(tdt) )

0.004879635653871178

### При помощи GridSearchCV произвести подбор гиперпараметров для поиска наилучшего леса

In [33]:
forest_params = {
    'max_depth': range(1,4),
    'max_features': range(3,5)
}
#подбираем параметры 
#чем больший охват у параметров, тем дольше работает алгоритм, поэтому надо аккуратнее подбирать
forest_grid = sklearn.model_selection.GridSearchCV(forest, forest_params, cv = 5)

forest_grid.fit( tdtr, y_train )

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(n_estimators=20, random_state=17),
             param_grid={'max_depth': range(1, 4), 'max_features': range(3, 5)})

### Посмотрм на определенные параметры

In [34]:
forest_grid.best_params_, forest_grid.best_score_ 

({'max_depth': 1, 'max_features': 3}, 0.0031453362255965287)

### Посмотрим на долю правильных ответов

In [35]:
metr.accuracy_score( y_test, forest_grid.predict(tdt) ) 

0.002602472348731295